In [ ]:
# 01 
from PyQt5.QtWidgets import *
import sys
import winsound

class BeepSound(QMainWindow):
    def __init__(self):
        # QMainWindow 초기화 (부모, 플래그 전달)
        super().__init__()
        
        # [1] 윈도우 기본 설정
        self.setWindowTitle('삑 소리 내기')
        self.setGeometry(200, 200, 500, 100)
        
        # [2] 버튼 3개 생성
        soundButton = QPushButton('소리', self)
        quitButton = QPushButton('나가기', self)
        
        # [3] 안내 라벨
        self.label = QLabel('환영합니다!', self)
        
        # [4] 버튼 위치 배치
        soundButton.setGeometry(10, 10, 100, 30)
        quitButton.setGeometry(210, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 70)
        
        # [5] 버튼 클릭 연결 (Signal-Slot 연결)
        soundButton.clicked.connect(self.soundFunction)
        quitButton.clicked.connect(self.quitFunction)
        
    # [6] 짧은 삑 소리 함수
    def soundFunction(self):
        winsound.PlaySound("SystemQuestion", winsound.SND_ALIAS)


    # [8] 종료 함수
    def quitFunction(self):
        self.close()

# [9] PyQt5 애플리케이션 실행 루프
app = QApplication(sys.argv)
win = BeepSound()
win.show()
app.exec_()

0

In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QTimer
import sys
import cv2 as cv
import numpy as np

class VideoCaptureApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('QTimer로 비디오 프레임 수집 개선 버전')
        self.setGeometry(200, 200, 800, 100)

        # 버튼들
        self.videoButton = QPushButton('비디오 켜기', self)
        self.captureButton = QPushButton('프레임 잡기', self)
        self.multiCaptureButton = QPushButton('여러 프레임 잡기', self)
        self.stackShowButton = QPushButton('프레임 이어보기', self)
        self.quitButton = QPushButton('나가기', self)

        # 버튼 배치
        self.videoButton.setGeometry(10, 10, 120, 30)
        self.captureButton.setGeometry(140, 10, 120, 30)
        self.multiCaptureButton.setGeometry(270, 10, 120, 30)
        self.stackShowButton.setGeometry(400, 10, 150, 30)
        self.quitButton.setGeometry(560, 10, 100, 30)

        # 버튼 연결
        self.videoButton.clicked.connect(self.startVideo)
        self.captureButton.clicked.connect(self.captureFrame)
        self.multiCaptureButton.clicked.connect(self.startMultiCapture)
        self.stackShowButton.clicked.connect(self.showStack)
        self.quitButton.clicked.connect(self.quitApp)

        # 초기 비활성화
        self.captureButton.setEnabled(False)
        self.multiCaptureButton.setEnabled(False)
        self.stackShowButton.setEnabled(False)

        # 내부 변수들
        self.cap = None
        self.timer = QTimer()
        self.timer.timeout.connect(self.updateFrame)

        self.frame = None
        self.capturedFrame = None
        self.frames = []
        self.multiCaptureMode = False

    def startVideo(self):
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            print('카메라 연결 실패')
            self.close()
            return

        self.timer.start(30)  # QTimer 시작
        print("비디오 스트림 시작")

        # 버튼 상태 변경
        self.captureButton.setEnabled(True)
        self.multiCaptureButton.setEnabled(True)
        self.stackShowButton.setEnabled(True)

    def updateFrame(self):
        ret, self.frame = self.cap.read()
        if not ret:
            return

        cv.imshow('Video Stream', self.frame)

        key = cv.waitKey(1)
        if self.multiCaptureMode and key == ord('c'):
            self.frames.append(self.frame.copy())
            print(f"프레임 {len(self.frames)} 저장됨")

    def captureFrame(self):
        if self.frame is not None:
            self.capturedFrame = self.frame.copy()
            cv.imshow('Captured Frame', self.capturedFrame)
            print("단일 프레임 저장 완료!")
        else:
            print("캡처할 프레임이 없습니다!")

    def startMultiCapture(self):
        if self.cap is None or not self.cap.isOpened():
            print("비디오를 먼저 켜세요!")
            return

        self.frames = []
        self.multiCaptureMode = True
        print("여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료")

    def showStack(self):
        if len(self.frames) == 0:
            print("저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.")
            return

        stack = cv.resize(self.frames[0], dsize=(0, 0), fx=0.3, fy=0.3)
        for i in range(1, len(self.frames)):
            resized = cv.resize(self.frames[i], dsize=(0, 0), fx=0.3, fy=0.3)
            stack = np.hstack((stack, resized))

        cv.imshow('Stacked Frames', stack)
        print(f"총 {len(self.frames)}개의 프레임 이어서 표시됨")

    def quitApp(self):
        self.timer.stop()
        if self.cap is not None:
            self.cap.release()
        cv.destroyAllWindows()
        self.close()


if __name__ == '__main__':
    app = QApplication(sys.argv)
    win = VideoCaptureApp()
    win.show()
    app.exec_()



비디오 스트림 시작
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.
단일 프레임 저장 완료!
단일 프레임 저장 완료!
단일 프레임 저장 완료!
저장된 프레임이 없습니다! 먼저 여러 프레임을 잡으세요.
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
단일 프레임 저장 완료!
여러 프레임 잡기 모드 활성화: c 키를 눌러 프레임 저장, q 키로 종료
프레임 1 저장됨
프레임 2 저장됨
프레임 3 저장됨
총 3개의 프레임 이어서 표시됨


In [ ]:
#03 예외처리
import cv2 as cv 
import numpy as np
import sys
from PyQt5.QtWidgets import *

# [1] 메인 윈도우 클래스 정의
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 초기화

        self.setWindowTitle('오림')  # 윈도우 제목
        self.setGeometry(200, 200, 700, 200)  # 위치(x,y)와 크기(width, height)

        # [2] 버튼 7개 생성
        fileButton = QPushButton('파일', self)
        paintButton = QPushButton('페인팅', self)
        cutButton = QPushButton('오림', self)
        incButton = QPushButton('+', self)
        decButton = QPushButton('-', self)
        saveButton = QPushButton('저장', self)
        quitButton = QPushButton('나가기', self)

        # [3] 버튼 위치 배치
        fileButton.setGeometry(10, 10, 100, 30)
        paintButton.setGeometry(110, 10, 100, 30)
        cutButton.setGeometry(210, 10, 100, 30)
        incButton.setGeometry(310, 10, 50, 30)
        decButton.setGeometry(360, 10, 50, 30)
        saveButton.setGeometry(410, 10, 100, 30)
        quitButton.setGeometry(510, 10, 100, 30)

        # [4] 버튼 클릭 시 실행될 함수 연결 (Signal-Slot)
        fileButton.clicked.connect(self.fileOpenFunction)
        paintButton.clicked.connect(self.paintFunction)
        cutButton.clicked.connect(self.cutFunction)
        incButton.clicked.connect(self.incFunction)
        decButton.clicked.connect(self.decFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

        # [5] 페인팅 기본값
        self.BrushSiz = 5                   # 붓 크기 초기값
        self.LColor, self.RColor = (255, 0, 0), (0, 0, 255)  # 파랑=물체, 빨강=배경

    # [6] 파일 열기 함수
    def fileOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', './')  # 파일 선택
        self.img = cv.imread(fname[0])  # 이미지 읽기
        if self.img is None:
            sys.exit('파일을 찾을 수 없습니다.')

        self.img_show = np.copy(self.img)  # 표시용 이미지 (붓칠 표시)
        cv.imshow('Painting', self.img_show)

        # GrabCut용 마스크 초기화
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv.GC_PR_BGD  # 모든 화소를 잠재적 배경으로 초기화

    # [7] 페인팅 함수: 마우스 이벤트 콜백 연결
    def paintFunction(self):
        cv.setMouseCallback('Painting', self.painting)

    # [8] 실제 마우스 드로잉 함수
    def painting(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            # 왼쪽 버튼 클릭 → 파란색 → 물체 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)  # 확실한 전경

        elif event == cv.EVENT_RBUTTONDOWN:
            # 오른쪽 버튼 클릭 → 빨간색 → 배경 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)  # 확실한 배경

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
            # 왼쪽 버튼 누른 채 이동 → 파란색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
            # 오른쪽 버튼 누른 채 이동 → 빨간색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)

        cv.imshow('Painting', self.img_show)

    # [9] GrabCut으로 전경/배경 분리 (오림)
    def cutFunction(self):
        # ① 이미지 로드 여부 확인
        if not hasattr(self, 'img') or self.img is None:
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오세요!')
            return

        # ② 마스크 초기화 여부 확인
        if not hasattr(self, 'mask') or self.mask is None:
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오고 붓칠을 하세요!')
            return
        
        num_fg = np.count_nonzero(self.mask == cv.GC_FGD)
        num_bg = np.count_nonzero(self.mask == cv.GC_BGD)

        if num_fg == 0 or num_bg == 0:
            QMessageBox.warning(self, '경고', '붓으로 전경(파랑)과 배경(빨강)을 모두 지정해야 합니다!')
            return
    
        background = np.zeros((1, 65), np.float64)  # 배경 GMM
        foreground = np.zeros((1, 65), np.float64)  # 전경 GMM

        cv.grabCut(
            self.img,         # 원본 이미지
            self.mask,        # 사용자 붓칠 마스크
            None,             # ROI 없음
            background,
            foreground,
            5,                # 반복 횟수
            cv.GC_INIT_WITH_MASK  # 마스크 기반으로 초기화
        )

        # 확실한/잠재적 배경은 0, 나머지는 1로
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype('uint8')
        self.grabImg = self.img * mask2[:, :, np.newaxis]  # 전경만 남김

        cv.imshow('Scissoring', self.grabImg)  # 결과 출력

    # [10] 붓 크기 증가
    def incFunction(self):
        self.BrushSiz = min(20, self.BrushSiz + 1)

    # [11] 붓 크기 감소
    def decFunction(self):
        self.BrushSiz = max(1, self.BrushSiz - 1)

    # [12] 결과 이미지 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.grabImg)

    # [13] 종료
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

# [14] PyQt5 앱 실행
app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

0

In [ ]:
# 04 색칠 지우기
import cv2 as cv 
import numpy as np
import sys
from PyQt5.QtWidgets import *

# [1] 메인 윈도우 클래스 정의
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 초기화

        self.setWindowTitle('오림')  # 윈도우 제목
        self.setGeometry(200, 200, 750, 200)  # 위치(x,y)와 크기(width, height)

        # [2] 버튼 7개 생성
        fileButton = QPushButton('파일', self)
        paintButton = QPushButton('페인팅', self)
        clearButton = QPushButton('지우기', self)
        cutButton = QPushButton('오림', self)
        incButton = QPushButton('+', self)
        decButton = QPushButton('-', self)
        saveButton = QPushButton('저장', self)
        quitButton = QPushButton('나가기', self)

        # [3] 버튼 위치 배치
        fileButton.setGeometry(10, 10, 100, 30)
        paintButton.setGeometry(110, 10, 100, 30)
        clearButton.setGeometry(210, 10, 100, 30)
        cutButton.setGeometry(310, 10, 100, 30)
        incButton.setGeometry(410, 10, 50, 30)
        decButton.setGeometry(470, 10, 50, 30)
        saveButton.setGeometry(510, 10, 100, 30)
        quitButton.setGeometry(610, 10, 100, 30)

        # [4] 버튼 클릭 시 실행될 함수 연결 (Signal-Slot)
        fileButton.clicked.connect(self.fileOpenFunction)
        paintButton.clicked.connect(self.paintFunction)
        clearButton.clicked.connect(self.clearPainting)
        cutButton.clicked.connect(self.cutFunction)
        incButton.clicked.connect(self.incFunction)
        decButton.clicked.connect(self.decFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

        # [5] 페인팅 기본값
        self.BrushSiz = 5                   # 붓 크기 초기값
        self.LColor, self.RColor = (255, 0, 0), (0, 0, 255)  # 파랑=물체, 빨강=배경

    # [6] 파일 열기 함수
    def fileOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, 'Open file', './')  # 파일 선택
        self.img = cv.imread(fname[0])  # 이미지 읽기
        if self.img is None:
            sys.exit('파일을 찾을 수 없습니다.')

        self.img_show = np.copy(self.img)  # 표시용 이미지 (붓칠 표시)
        cv.imshow('Painting', self.img_show)

        # GrabCut용 마스크 초기화
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv.GC_PR_BGD  # 모든 화소를 잠재적 배경으로 초기화

    # [7] 페인팅 함수: 마우스 이벤트 콜백 연결
    def paintFunction(self):
        cv.setMouseCallback('Painting', self.painting)

    # [8] 실제 마우스 드로잉 함수
    def painting(self, event, x, y, flags, param):
        if event == cv.EVENT_LBUTTONDOWN:
            # 왼쪽 버튼 클릭 → 파란색 → 물체 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)  # 확실한 전경

        elif event == cv.EVENT_RBUTTONDOWN:
            # 오른쪽 버튼 클릭 → 빨간색 → 배경 영역
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)  # 확실한 배경

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
            # 왼쪽 버튼 누른 채 이동 → 파란색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.LColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_FGD, -1)

        elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
            # 오른쪽 버튼 누른 채 이동 → 빨간색 그리기
            cv.circle(self.img_show, (x, y), self.BrushSiz, self.RColor, -1)
            cv.circle(self.mask, (x, y), self.BrushSiz, cv.GC_BGD, -1)

        cv.imshow('Painting', self.img_show)
        
    def clearPainting(self):
        if not hasattr(self, 'img') or self.img is None:
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오세요!')
            return

        # 표시용 이미지 원본으로 초기화
        self.img_show = np.copy(self.img)

        # GrabCut 마스크 초기화 (모두 잠재적 배경으로)
        self.mask[:, :] = cv.GC_PR_BGD

        cv.imshow('Painting', self.img_show)
        
    # [9] GrabCut으로 전경/배경 분리 (오림)
    def cutFunction(self):
        # ① 이미지 로드 여부 확인
        if not hasattr(self, 'img') or self.img is None:
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오세요!')
            return

        # ② 마스크 초기화 여부 확인
        if not hasattr(self, 'mask') or self.mask is None:
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오고 붓칠을 하세요!')
            return
        
        num_fg = np.count_nonzero(self.mask == cv.GC_FGD)
        num_bg = np.count_nonzero(self.mask == cv.GC_BGD)

        if num_fg == 0 or num_bg == 0:
            QMessageBox.warning(self, '경고', '붓으로 전경(파랑)과 배경(빨강)을 모두 지정해야 합니다!')
            return
    
        background = np.zeros((1, 65), np.float64)  # 배경 GMM
        foreground = np.zeros((1, 65), np.float64)  # 전경 GMM

        cv.grabCut(
            self.img,         # 원본 이미지
            self.mask,        # 사용자 붓칠 마스크
            None,             # ROI 없음
            background,
            foreground,
            5,                # 반복 횟수
            cv.GC_INIT_WITH_MASK  # 마스크 기반으로 초기화
        )

        # 확실한/잠재적 배경은 0, 나머지는 1로
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype('uint8')
        self.grabImg = self.img * mask2[:, :, np.newaxis]  # 전경만 남김

        cv.imshow('Scissoring', self.grabImg)  # 결과 출력

    # [10] 붓 크기 증가
    def incFunction(self):
        self.BrushSiz = min(20, self.BrushSiz + 1)

    # [11] 붓 크기 감소
    def decFunction(self):
        self.BrushSiz = max(1, self.BrushSiz - 1)

    # [12] 결과 이미지 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.grabImg)

    # [13] 종료
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

# [14] PyQt5 앱 실행
app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

0

In [1]:
# 05
from PyQt5.QtWidgets import *   # PyQt5 위젯
import cv2 as cv                # OpenCV
import numpy as np              # Numpy
import winsound                 # Windows beep sound
import sys                      # 시스템 종료용

# [1] 파노라마 GUI 메인 클래스
class Panorama(QMainWindow):
    def __init__(self):
        super().__init__()  # QMainWindow 초기화
        self.setWindowTitle('파노라마 영상')     # 창 제목
        self.setGeometry(200, 200, 700, 200)     # 창 위치(x,y) + 크기(width,height)

        # [2] 버튼 5개 + 라벨 생성
        collectButton = QPushButton('영상 수집', self)
        self.showButton = QPushButton('영상 보기', self)
        self.stitchButton = QPushButton('봉합', self)
        cropButton = QPushButton('검은색 영역 제거', self)
        self.saveButton = QPushButton('저장', self)
        quitButton = QPushButton('나가기', self)
        self.label = QLabel('환영합니다!', self)

        # [3] 버튼 & 라벨 위치 설정
        collectButton.setGeometry(10, 25, 100, 30)
        self.showButton.setGeometry(110, 25, 100, 30)
        self.stitchButton.setGeometry(210, 25, 100, 30)
        cropButton.setGeometry(210, 60, 100, 30)
        self.saveButton.setGeometry(310, 25, 100, 30)
        quitButton.setGeometry(450, 25, 100, 30)
        self.label.setGeometry(10, 70, 600, 170)

        # [4] 수집, 봉합, 저장 버튼은 처음엔 비활성화
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)

        # [5] 버튼 클릭 시 함수 연결 (Signal-Slot)
        collectButton.clicked.connect(self.collectFunction)
        self.showButton.clicked.connect(self.showFunction)
        self.stitchButton.clicked.connect(self.stitchFunction)
        cropButton.clicked.connect(self.cropFunction)
        self.saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

    # [6] 영상 수집 함수
    def collectFunction(self):
        # 수집 전 버튼 다시 초기화
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)

        # 사용자에게 안내
        self.label.setText('c를 여러 번 눌러 수집하고 끝나면 q를 눌러 비디오를 끕니다.')

        # 웹캠 열기 (DirectShow 모드)
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            sys.exit('카메라 연결 실패')

        self.imgs = []  # 영상 저장 리스트

        # 비디오 루프
        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            cv.imshow('video display', frame)  # 현재 프레임 표시

            key = cv.waitKey(1)
            if key == ord('c'):
                self.imgs.append(frame)  # c 누르면 현재 프레임 저장
                print(self.imgs)
            elif key == ord('q'):
                self.cap.release()
                cv.destroyWindow('video display')
                break

        # 2장 이상이면 나머지 버튼 활성화
        if len(self.imgs) >= 2:
            self.showButton.setEnabled(True)
            self.stitchButton.setEnabled(True)
            self.saveButton.setEnabled(True)

    # [7] 수집된 영상 보기
    def showFunction(self):
        self.label.setText('수집된 영상은 ' + str(len(self.imgs)) + '장 입니다.')

        # 첫 번째 영상은 스택 시작점 (크기 축소)
        stack = cv.resize(self.imgs[0], dsize=(0, 0), fx=0.25, fy=0.25)

        # 나머지 영상 가로로 이어붙이기
        for i in range(1, len(self.imgs)):
            resized = cv.resize(self.imgs[i], dsize=(0, 0), fx=0.25, fy=0.25)
            stack = np.hstack((stack, resized))

        cv.imshow('Image collection', stack)  # 이어붙인 결과 표시

    # [8] 영상 Stitching 함수 (파노라마 봉합)
    def stitchFunction(self):
        stitcher = cv.Stitcher_create()  # OpenCV 파노라마 스티쳐 객체 생성
        status, self.img_stitched = stitcher.stitch(self.imgs)  # 이미지 리스트 봉합 시도

        if status == cv.STITCHER_OK:
            # 성공: 파노라마 출력
            cv.imshow('Image stitched panorama', self.img_stitched)
        else:
            # 실패: 경고음 + 라벨 안내
            winsound.Beep(3000, 500)
            self.label.setText('파노라마 제작에 실패했습니다. 다시 시도하세요.')
    
    def cropFunction(self):
        if not hasattr(self, 'img_stitched'):
            QMessageBox.warning(self, '경고', '먼저 파노라마 봉합을 실행하세요!')
            return

        # 흑백으로 마스크 생성
        gray = cv.cvtColor(self.img_stitched, cv.COLOR_BGR2GRAY)
        _, thresh = cv.threshold(gray, 1, 255, cv.THRESH_BINARY)

        # 컨투어 찾기
        contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

        if len(contours) == 0:
            QMessageBox.warning(self, '경고', '잘라낼 수 있는 영역이 없습니다.')
            return

        # 모든 컨투어의 바운딩 박스를 합침
        all_points = np.vstack(contours)
        x, y, w, h = cv.boundingRect(all_points)

        # Crop
        cropped = self.img_stitched[y:y+h, x:x+w]

        # 다시 흑백 마스크 생성
        gray_crop = cv.cvtColor(cropped, cv.COLOR_BGR2GRAY)
        _, mask_crop = cv.threshold(gray_crop, 1, 255, cv.THRESH_BINARY)

        # 실제 영역만 마스킹 → 남은 검은 부분까지 제거
        result = cv.bitwise_and(cropped, cropped, mask=mask_crop)

        self.img_stitched = result
        cv.imshow('Cropped Panorama', self.img_stitched)
        self.label.setText('검은 영역이 제거되었습니다.')
    
    # [9] 스티칭 결과 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.img_stitched)

    # [10] 종료 함수
    def quitFunction(self):
        self.cap.release()  # 카메라 닫기
        cv.destroyAllWindows()  # 모든 OpenCV 창 닫기
        self.close()  # PyQt 윈도우 닫기

# [11] PyQt5 앱 실행 루프
app = QApplication(sys.argv)
win = Panorama()
win.show()
app.exec_()

[array([[[141, 174, 169],
        [141, 174, 169],
        [138, 172, 171],
        ...,
        [ 55,  60,  51],
        [ 59,  60,  51],
        [ 61,  62,  53]],

       [[137, 175, 170],
        [137, 175, 170],
        [139, 173, 172],
        ...,
        [ 58,  61,  52],
        [ 57,  58,  54],
        [ 59,  60,  56]],

       [[141, 176, 172],
        [139, 174, 170],
        [139, 174, 170],
        ...,
        [ 57,  59,  53],
        [ 56,  59,  55],
        [ 57,  60,  56]],

       ...,

       [[ 99, 133, 166],
        [ 99, 133, 166],
        [ 99, 133, 164],
        ...,
        [ 23,  43,  49],
        [ 21,  40,  48],
        [ 23,  42,  50]],

       [[ 97, 134, 164],
        [ 97, 134, 164],
        [ 99, 133, 164],
        ...,
        [ 25,  43,  49],
        [ 22,  39,  49],
        [ 23,  40,  50]],

       [[ 98, 135, 167],
        [ 97, 134, 166],
        [ 99, 133, 164],
        ...,
        [ 23,  41,  47],
        [ 22,  39,  49],
        [ 22,  39,  49]

0

In [1]:
# 06 블러 추가

import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys

# [1] PyQt5 메인 윈도우 클래스
class SpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('사진 특수 효과')
        self.setGeometry(200, 200, 800, 200)

        # [2] 버튼 + 콤보박스 + 라벨 생성
        pictureButton = QPushButton('사진 읽기', self)
        embossButton = QPushButton('엠보싱', self)
        cartoonButton = QPushButton('카툰', self)
        sketchButton = QPushButton('연필 스케치', self)
        oilButton = QPushButton('유화', self)
        blurButton = QPushButton('블러 적용', self) 
        saveButton = QPushButton('저장하기', self)

        # 효과 선택 콤보박스 (저장용)
        self.pickCombo = QComboBox(self)
        self.pickCombo.addItems([
            '엠보싱', '카툰', '연필 스케치(명암)', '연필 스케치(컬러)', '유화', '블러'
        ])

        # 블러 강도 선택 콤보박스 
        self.blurCombo = QComboBox(self)
        self.blurCombo.addItems(['블러(약하게)', '블러(적절하게)', '블러(강하게)'])

        quitButton = QPushButton('나가기', self)
        self.label = QLabel('환영합니다!', self)

        # [3] 위치 배치
        pictureButton.setGeometry(10, 10, 100, 30)
        embossButton.setGeometry(110, 10, 100, 30)
        cartoonButton.setGeometry(210, 10, 100, 30)
        sketchButton.setGeometry(310, 10, 100, 30)
        oilButton.setGeometry(410, 10, 100, 30)
        blurButton.setGeometry(510, 10, 100, 30)    
        saveButton.setGeometry(620, 10, 100, 30)

        self.pickCombo.setGeometry(620, 40, 100, 30)   # 저장용 효과 선택
        self.blurCombo.setGeometry(510, 40, 100, 30)   

        quitButton.setGeometry(730, 10, 60, 30)
        self.label.setGeometry(10, 70, 500, 120)

        # [4] 시그널-슬롯 연결
        pictureButton.clicked.connect(self.pictureOpenFunction)
        embossButton.clicked.connect(self.embossFunction)
        cartoonButton.clicked.connect(self.cartoonFunction)
        sketchButton.clicked.connect(self.sketchFunction)
        oilButton.clicked.connect(self.oilFunction)
        blurButton.clicked.connect(self.blurButtonClicked)  
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)

    # [5] 이미지 읽기
    def pictureOpenFunction(self):
        fname = QFileDialog.getOpenFileName(self, '사진 읽기', './')
        self.img = cv.imread(fname[0])
        if self.img is None:
            sys.exit('파일을 찾을 수 없습니다.')
        cv.imshow('Original', self.img)

    # [6] 엠보싱
    def embossFunction(self):
        femboss = np.array([[-1.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0],
                            [0.0, 0.0, 1.0]])
        gray = cv.cvtColor(self.img, cv.COLOR_BGR2GRAY)
        gray16 = np.int16(gray)
        self.emboss = np.uint8(np.clip(cv.filter2D(gray16, -1, femboss) + 128, 0, 255))
        cv.imshow('Emboss', self.emboss)

    # [7] 카툰
    def cartoonFunction(self):
        self.cartoon = cv.stylization(self.img, sigma_s=60, sigma_r=0.45)
        cv.imshow('Cartoon', self.cartoon)

    # [8] 연필 스케치
    def sketchFunction(self):
        self.sketch_gray, self.sketch_color = cv.pencilSketch(
            self.img, sigma_s=60, sigma_r=0.07, shade_factor=0.02
        )
        cv.imshow('Pencil Sketch (Gray)', self.sketch_gray)
        cv.imshow('Pencil Sketch (Color)', self.sketch_color)

    # [9] 유화
    def oilFunction(self):
        self.oil = cv.xphoto.oilPainting(self.img, 10, 1, cv.COLOR_BGR2Lab)
        cv.imshow('Oil Painting', self.oil)

    # [10] 블러 전용 버튼 클릭 시
    def blurButtonClicked(self):
        if not hasattr(self, 'img'):
            QMessageBox.warning(self, '경고', '먼저 이미지를 불러오세요!')
            return
        level = self.blurCombo.currentText()
        self.blurFunction(level)

    # [11] 블러 함수
    def blurFunction(self, level):
        if level == '블러(약하게)':
            ksize = 5
        elif level == '블러(적절하게)':
            ksize = 15
        elif level == '블러(강하게)':
            ksize = 25
        else:
            ksize = 5

        kernel = np.zeros((ksize, ksize))
        kernel[int((ksize - 1)/2), :] = np.ones(ksize)
        kernel = kernel / ksize

        self.blur = cv.filter2D(self.img, -1, kernel)
        cv.imshow('Motion Blur', self.blur)

    # [12] 저장
    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        i = self.pickCombo.currentIndex()

        if i == 0:
            cv.imwrite(fname[0], self.emboss)
        elif i == 1:
            cv.imwrite(fname[0], self.cartoon)
        elif i == 2:
            cv.imwrite(fname[0], self.sketch_gray)
        elif i == 3:
            cv.imwrite(fname[0], self.sketch_color)
        elif i == 4:
            cv.imwrite(fname[0], self.oil)
        elif i == 5:
            # 저장 시에도 강도 선택 반영
            level = self.blurCombo.currentText()
            self.blurFunction(level)  # 블러 다시 적용 후 저장
            cv.imwrite(fname[0], self.blur)

    # [13] 종료
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

# [14] PyQt5 앱 실행
app = QApplication(sys.argv)
win = SpecialEffect()
win.show()
app.exec_()


0